- Try the variance approx with \Sigma = np.linalg.inv(glasso) and plug it directly into the graph to check out whether it does better than the glasso.

- It converges nicely. Need to find a way how to devise a method to test the perormance of these new estimators empirically beacuse this takes ages.


In [87]:
import sys, json
from datetime import datetime as dt
import numpy as np
import scipy
from preconditioners.utils import generate_c, generate_centered_gaussian_data
from preconditioners.cov_approx.variance_cov_approx import *
from sklearn.covariance import  GraphicalLasso

In [88]:
# parameters of the run
n_epochs = 100
iter_per_epoch = 500
tol = 0.0001
n = 100
d = 300
sigma2 = 1
ro = 0.5
regime = 'autoregressive'
regul_lambda = 0 # for log barrier use 0.001
lr_start = 0.5
lr_decay = 0.98

params = {
    'n_epochs' : n_epochs,
    'iter_per_epoch' : iter_per_epoch,
    'tol' : tol,
    'n' : n,
    'd' : d,
    'sigma2' : sigma2,
    'ro' : ro,
    'regime' : regime,
    'regul_lambda' : regul_lambda,
    'lr_start' : lr_start,
    'lr_decay' : lr_decay
}

In [89]:
# generate data and initialization
c = generate_c(ro=ro,
                regime=regime,
                n=n,
                d=d
                )
w_star = np.random.multivariate_normal(mean=np.zeros(d), cov=np.eye(d))
X, y, xi = generate_centered_gaussian_data(w_star,
                                            c,
                                            n=n,
                                            d=d,
                                            sigma2=sigma2,
                                            fix_norm_of_x=False)

# initialize C (cholesky), cov_inv, regul_lambda and learning rate
# is this a good way to initialize?
cov_empir = X.T.dot(X) / n
cov_inv = np.linalg.inv(cov_empir + 0.1 * np.eye(d))
gl = GraphicalLasso(assume_centered=True, alpha=0.25, tol=1e-4).fit(X)
cov_gl = gl.covariance_
C = scipy.linalg.cholesky(cov_inv) + 0.5 * generate_c(ro=0.2,
                                                        regime='autoregressive',
                                                        n=n,
                                                        d=d,
                                                        )
    

/Users/eduardoravkin/Dropbox/My Mac (Eduard’s MacBook Pro)/Desktop/ml_research/PhD/preconditioners/src/preconditioners/utils.py:219: UserWarning: Warning, norms of datapoints are not sqrt(d)
  warnings.warn('Warning, norms of datapoints are not sqrt(d)')


In [90]:
# run optimization
for epoch in range(n_epochs):
    if epoch == 0:
        lr = lr_start
    else:
        lr = lr * lr_decay

    for i in range(iter_per_epoch):
        # compute loss and gradient
        cov_inv = C.dot(C.T)
        loss_val, grad_loss_val = fAndG_invbarrier(
            B = cov_gl,
            C = C,
            X = X,
            a = regul_lambda
        )
        error = np.linalg.norm(grad_loss_val)

        # update C (- because we are maximizing)
        C = C - lr * grad_loss_val
        # update regul_lambda
        # regul_lambda = regul_lambda - lr*np.trace(grad_loss_val.dot(grad_loss_val.T))
        # check if we are done
        if i % 50 == 0:
            print(f"iteration {i}/{iter_per_epoch} of epoch {epoch}/{n_epochs}, loss {loss_val} and error {error}")
        if error < tol:
            break
    if error < tol:
        break

dtstamp = str(dt.now()).replace(' ', '_')
with open(f'results_{dtstamp}.json', 'w') as f:
    json.dump({'C' : C.tolist(), 'loss' : float(loss_val), 'error' : float(error), 'parans' : params}, f)

iteration 0/500 of epoch 0/100, loss 1.2356784562484702 and error 0.14041451922415543
iteration 50/500 of epoch 0/100, loss 0.9754877248832658 and error 0.07544548437373999
iteration 100/500 of epoch 0/100, loss 0.8745091490209388 and error 0.053543012752988194
iteration 150/500 of epoch 0/100, loss 0.8183241990435243 and error 0.041932059866995275
iteration 200/500 of epoch 0/100, loss 0.7819627053466467 and error 0.034649445833522294
iteration 250/500 of epoch 0/100, loss 0.7562722378283997 and error 0.02962794819679009
iteration 300/500 of epoch 0/100, loss 0.7370386816601551 and error 0.025942553595160334
iteration 350/500 of epoch 0/100, loss 0.7220346395564323 and error 0.023114045261372413
iteration 400/500 of epoch 0/100, loss 0.7099657297867435 and error 0.020868673990443624
iteration 450/500 of epoch 0/100, loss 0.7000252603808442 and error 0.019038580346164156
iteration 0/500 of epoch 1/100, loss 0.6916825588516993 and error 0.0175150906931161
iteration 50/500 of epoch 1/100

KeyboardInterrupt: 

In [91]:
c

array([[1.00000000e+00, 5.00000000e-01, 2.50000000e-01, ...,
        3.92727477e-90, 1.96363739e-90, 9.81818693e-91],
       [5.00000000e-01, 1.00000000e+00, 5.00000000e-01, ...,
        7.85454954e-90, 3.92727477e-90, 1.96363739e-90],
       [2.50000000e-01, 5.00000000e-01, 1.00000000e+00, ...,
        1.57090991e-89, 7.85454954e-90, 3.92727477e-90],
       ...,
       [3.92727477e-90, 7.85454954e-90, 1.57090991e-89, ...,
        1.00000000e+00, 5.00000000e-01, 2.50000000e-01],
       [1.96363739e-90, 3.92727477e-90, 7.85454954e-90, ...,
        5.00000000e-01, 1.00000000e+00, 5.00000000e-01],
       [9.81818693e-91, 1.96363739e-90, 3.92727477e-90, ...,
        2.50000000e-01, 5.00000000e-01, 1.00000000e+00]])

In [92]:
np.linalg.inv(c)

array([[ 1.33333333e+000, -6.66666667e-001,  0.00000000e+000, ...,
         2.90676725e-106,  1.45338363e-106,  0.00000000e+000],
       [-6.66666667e-001,  1.66666667e+000, -6.66666667e-001, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       [ 0.00000000e+000, -6.66666667e-001,  1.66666667e+000, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       ...,
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         1.66666667e+000, -6.66666667e-001,  0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
        -6.66666667e-001,  1.66666667e+000, -6.66666667e-001],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         0.00000000e+000, -6.66666667e-001,  1.33333333e+000]])

In [93]:
C.dot(C.T)

array([[ 9.02286385, -0.60023547, -0.43612159, ..., -0.56330828,
        -0.263471  ,  0.35776726],
       [-0.60023547, 11.278642  , -1.43656359, ..., -0.43187039,
         0.06942722,  0.16263109],
       [-0.43612159, -1.43656359,  9.16594444, ...,  0.24112757,
        -0.20680937,  0.09425723],
       ...,
       [-0.56330828, -0.43187039,  0.24112757, ...,  4.9541649 ,
        -0.22389379, -0.28919304],
       [-0.263471  ,  0.06942722, -0.20680937, ..., -0.22389379,
         4.57079474, -0.6826814 ],
       [ 0.35776726,  0.16263109,  0.09425723, ..., -0.28919304,
        -0.6826814 ,  4.19215567]])

In [94]:
np.linalg.inv(C.dot(C.T))

array([[ 0.15374154,  0.02095593,  0.01417211, ...,  0.02612471,
         0.01276738, -0.01190051],
       [ 0.02095593,  0.12719178,  0.03471095, ...,  0.01455229,
         0.00264128, -0.01148969],
       [ 0.01417211,  0.03471095,  0.15644763, ..., -0.01074425,
         0.01856873, -0.00813217],
       ...,
       [ 0.02612471,  0.01455229, -0.01074425, ...,  0.3510296 ,
        -0.01226485,  0.0341699 ],
       [ 0.01276738,  0.00264128,  0.01856873, ..., -0.01226485,
         0.35823919,  0.02833474],
       [-0.01190051, -0.01148969, -0.00813217, ...,  0.0341699 ,
         0.02833474,  0.45695548]])

In [95]:
cov_empir

array([[ 1.09071158,  0.38272015,  0.1797117 , ..., -0.22297014,
        -0.15155182, -0.02299876],
       [ 0.38272015,  0.84337385,  0.54547024, ..., -0.049755  ,
        -0.0547505 ,  0.03123735],
       [ 0.1797117 ,  0.54547024,  1.13406443, ..., -0.23063027,
        -0.29344909, -0.14604015],
       ...,
       [-0.22297014, -0.049755  , -0.23063027, ...,  1.07034783,
         0.55572373,  0.41349008],
       [-0.15155182, -0.0547505 , -0.29344909, ...,  0.55572373,
         1.34957272,  0.84196009],
       [-0.02299876,  0.03123735, -0.14604015, ...,  0.41349008,
         0.84196009,  1.4159501 ]])

In [96]:
gl = GraphicalLasso(assume_centered=True, alpha=0.25, tol=1e-4).fit(X)

In [97]:
gl.precision_

array([[ 0.94537365, -0.14709722, -0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14709722,  1.32796415, -0.33995816, ...,  0.        ,
         0.        , -0.        ],
       [-0.        , -0.33995816,  1.0498066 , ...,  0.        ,
         0.02798588,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.00671681,
        -0.21531002, -0.02540652],
       [ 0.        ,  0.        ,  0.02798588, ..., -0.21531002,
         0.97516427, -0.36484121],
       [ 0.        , -0.        ,  0.        , ..., -0.02540652,
        -0.36484121,  0.89119774]])

In [98]:
print(np.linalg.norm(C.dot(C.T) - np.linalg.inv(c)))
print(np.linalg.norm(C.dot(C.T)/10 - np.linalg.inv(c)))
print(np.linalg.norm(gl.precision_ - np.linalg.inv(c)))
print(np.linalg.norm(np.eye(d) - np.linalg.inv(c)))

146.5606126388698
20.84068729228886
13.646749931053467
19.961073228773156


In [99]:
print(np.linalg.norm(np.diag(C.dot(C.T)/10) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(gl.precision_) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(np.eye(d)) - np.diag(np.linalg.inv(c))))

13.765219425856607
8.838883486920599
11.518101695447337


In [100]:
print(np.diag(gl.precision_)[:10])
print(np.diag(C.dot(C.T)-9)[:10])
print(np.diag(np.linalg.inv(c))[:10])

[0.94537365 1.32796415 1.0498066  1.34333605 1.08850605 1.19844678
 0.93786783 1.14786484 1.3234459  1.09749534]
[0.02286385 2.278642   0.16594444 2.17422783 0.76256586 1.41127553
 0.27852307 0.87776483 1.73636637 0.95083447]
[1.33333333 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667]


In [101]:
from neurips_code.utils import *
from neurips_code.predicted_risk import *
from neurips_code.Regressor import LinearRegressor

c_e = generate_c_empir(X, empir='gl', alpha=0.25)
m = np.eye(d)
snr = 1
include_best_achievable_empirical = True
print('covariance matrix estimated')

# initialize models
reg_2 = LinearRegressor()
reg_c = LinearRegressor()
reg_ce = LinearRegressor()

# generate predictors
# matrix specifies which mirror descent we are using (GD if None)
reg_2.fit(X, y, matrix = None)
reg_c.fit(X, y, matrix = c)
reg_ce.fit(X, y, matrix = c_e)

w_a = compute_best_achievable_interpolator(X, y, c, m, snr)
reg_a = LinearRegressor(init = w_a)

if include_best_achievable_empirical:
    w_ae = compute_best_achievable_interpolator(X, y, c = c_e, m = np.eye(d), snr = 1, crossval_param = 10)
    reg_ae = LinearRegressor(init = w_ae)
    print('empirical approximation computed')

# best possible linear predictor
c_mhalf = np.linalg.inv(scipy.linalg.sqrtm(c)) # inverse square root of the covariance matrix
w_b = c_mhalf.dot( np.linalg.lstsq( X.dot(c_mhalf),  xi, rcond=None)[0] ) + w_star # best possible predictor
reg_b = LinearRegressor(init = w_b)

# calculate the expected risks
risk_2 = calculate_risk(w_star, c, reg_2.w ) + sigma2
risk_c = calculate_risk(w_star, c, reg_c.w) + sigma2
risk_ce = calculate_risk(w_star, c, reg_ce.w) + sigma2
risk_a = calculate_risk(w_star, c, reg_a.w) + sigma2
if include_best_achievable_empirical:
    risk_ae = calculate_risk(w_star, c, reg_ae.w) + sigma2
risk_b = calculate_risk(w_star, c, reg_b.w) + sigma2

risks = risk_2, risk_c, risk_ce, risk_a, risk_ae, risk_b

covariance matrix estimated
empirical approximation computed


In [102]:
risks

(127.15542471499435,
 171.79238056771385,
 135.15066024335806,
 139.05702715209173,
 127.18340808079245,
 1.4403526674118354)

In [103]:
from neurips_code.utils import *
from neurips_code.predicted_risk import *
from neurips_code.Regressor import LinearRegressor

new_P = C.dot(C.T)
m = np.eye(d)
snr = 1
include_best_achievable_empirical = True
print('covariance matrix estimated')

# initialize models
reg_2 = LinearRegressor()
reg_c = LinearRegressor()
reg_ce = LinearRegressor()

# generate predictors
# matrix specifies which mirror descent we are using (GD if None)
reg_2.fit(X, y, matrix = None)
reg_c.fit(X, y, matrix = c)
reg_ce.fit(X, y, matrix = new_P)

w_a = compute_best_achievable_interpolator(X, y, c, m, snr)
reg_a = LinearRegressor(init = w_a)

if include_best_achievable_empirical:
    w_ae = compute_best_achievable_interpolator(X, y, c = new_P, m = np.eye(d), snr = 1, crossval_param = 10)
    reg_ae = LinearRegressor(init = w_ae)
    print('empirical approximation computed')

# best possible linear predictor
c_mhalf = np.linalg.inv(scipy.linalg.sqrtm(c)) # inverse square root of the covariance matrix
w_b = c_mhalf.dot( np.linalg.lstsq( X.dot(c_mhalf),  xi, rcond=None)[0] ) + w_star # best possible predictor
reg_b = LinearRegressor(init = w_b)

# calculate the expected risks
risk_2 = calculate_risk(w_star, c, reg_2.w ) + sigma2
risk_c = calculate_risk(w_star, c, reg_c.w) + sigma2
risk_ce = calculate_risk(w_star, c, reg_ce.w) + sigma2
risk_a = calculate_risk(w_star, c, reg_a.w) + sigma2
if include_best_achievable_empirical:
    risk_ae = calculate_risk(w_star, c, reg_ae.w) + sigma2
risk_b = calculate_risk(w_star, c, reg_b.w) + sigma2

risks = risk_2, risk_c, risk_ce, risk_a, risk_ae, risk_b

covariance matrix estimated
empirical approximation computed


In [104]:
risks

(127.15542471499435,
 171.79238056771385,
 142.45707730526016,
 139.05702715209173,
 133.7357486844477,
 1.4403526674118354)